<a href="https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/beta/omegafold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OmegaFold
for more details see: [Github](https://github.com/HeliXonProtein/OmegaFold), [Preprint](https://www.biorxiv.org/content/10.1101/2022.07.21.500999v1)


In [ ]:
#@markdown ##Install

%%bash
if [ ! -d OmegaFold ]; then
  git clone --quiet https://github.com/HeliXonProtein/OmegaFold.git
  cd OmegaFold
  pip -q install -r requirements.txt
  pip -q install py3Dmol
  apt-get install aria2 -qq > /dev/null
  aria2c -q -x 16 https://helixon.s3.amazonaws.com/release1.pt
  mkdir -p ~/.cache/omegafold_ckpt
  mv release1.pt ~/.cache/omegafold_ckpt/model.pt
fi

In [ ]:
%%time
#@markdown ##Run
import re

sequence = "GWSTELEKHREELKEFLKKEGITNVEIRIDNGRLEVRVEGGTERLKRFLEELRQKLEKKGYTVDIKIE" #@param {type:"string"}
sequence = re.sub("[^A-Z]", "", sequence.upper())
num_cycle = 10 #@param ["10", "1", "2", "4", "8", "16", "32"] {type:"raw"}

with open("tmp.fasta","w") as out:
  out.write(f">tmp\n{sequence}\n")
%shell python ./OmegaFold/main.py --num_cycle={num_cycle} tmp.fasta .

In [20]:
#@markdown ##Display
import py3Dmol
def show_pdb(pdb_str, show_sidechains=False, show_mainchains=False,
             color="pLDDT", chains=None, vmin=50, vmax=90,
             color_HP=False, size=(800,480), hbondCutoff=4.0,
             animate=False):
  
  if chains is None: chains = 1
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width=size[0], height=size[1])
  if animate:
    view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  else:
    view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  if color == "pLDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':vmin,'max':vmax}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  if show_sidechains:
    BB = ['C','O','N']
    HP = ["ALA","GLY","VAL","ILE","LEU","PHE","MET","PRO","TRP","CYS","TYR"]
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                  {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  view.zoomTo()
  if animate: view.animate()
  return view

color = "confidence" #@param ["confidence", "rainbow"]
if color == "confidence": color = "pLDDT"
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
show_pdb(open("tmp.pdb",'r').read(), color=color, show_sidechains=show_sidechains, show_mainchains=show_mainchains).show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title Download prediction
from google.colab import files
files.download(f'tmp.pdb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>